In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.widgets import TextBox
import numpy as np

In [ ]:
df_1 = pd.read_csv("heat_data_fig_1_python.csv").set_index("Country")
df_2 = pd.read_csv("heat_data_fig_2_python.csv").set_index("Country")


In [ ]:
font= 'Microsoft Yi Baiti'

text_props = {'fontsize': 25, 'fontname':font}
title_props =  {'fontsize': 40, 'fontname':font, 'verticalalignment':'baseline'} 
legend_props = {'size':18, 'family':font}
matplotlib.rcParams['font.family'] = font

face_color = "white"
pie_size = (14,14)
bar_size = (18,12)
RE_explode = [0,0,0,0,0.1,0.1,0.1,0.1]

In [ ]:

gas = '#8c564b'
oil = '#7f7f7f'
other_fossils = '#c7c7c7'
heat_pumps = '#c5b0d5'
direct_re = '#2ca02c'
district_heat = '#d62728'
electricity = '#1f77b4'
non_re_elec = '#ffbb78'
re_elec = '#98df8a'
re_DHC = '#9edae5'
ambient = '#17becf'


colors_1 = [gas, oil, other_fossils, heat_pumps, direct_re, district_heat, electricity]
colors_2 = [gas, oil, other_fossils, non_re_elec, re_elec, direct_re, re_DHC, ambient]

c_Dict = {'Fossil gas':'tab:blue', 'Oil':'tab:orange', 'Other fossil fuels':'tab:green', 
'Heat pumps':'tab:red', 'Direct renewables':'tab:purple', 'District heat':'tab:brown', 'Electricity':'tab:pink'
}


In [ ]:

colors = sns.color_palette("tab20")
palette = sns.color_palette("tab20", 20).as_hex()
print(palette)
palette

In [ ]:
df_2

In [ ]:
cmap = matplotlib.cm.get_cmap('tab20')
for i in range(cmap.N):
    rgba = cmap(i)
    print(f"Hex: {matplotlib.colors.rgb2hex(rgba)}")

In [ ]:
labels_1 = df_1.columns
labels_2 = df_2.columns


In [ ]:
df_1

In [ ]:
def plot_fig_1(country):
    fig = plt.figure(figsize = pie_size)
    plt.pie(df_1.loc[country], colors=colors_1, startangle=90, autopct=lambda p: '{:.0f}%'.format(round(p)) if p > 0 else '', textprops=text_props)
    plt.title(f"What fuels are used in {country}?", fontdict=title_props, y=0.95)
    plt.legend(labels=labels_1, prop=legend_props, loc = 8, ncol=3, edgecolor='white', framealpha=0)
    centre_circle = plt.Circle((0,0),0.70,fc=face_color)
    ax = fig.gca()
    fig.patch.set_facecolor(face_color)
    ax.add_artist(centre_circle)
    plt.tight_layout()
    plt.savefig(f"Exported/{country}_{'figure_1'}")
    plt.show()


In [ ]:
def plot_fig_2(country):
    fig = plt.figure(figsize = pie_size)
    plt.pie(df_2.loc[country], colors=colors_2, startangle=90, autopct=lambda p: '{:.0f}%'.format(round(p)) if p > 0 else '', textprops=text_props)
    plt.title(f"How clean is the heat in {country}?", fontdict=title_props, y=0.95)
    plt.legend(labels=labels_2, prop=legend_props, loc = 8, ncol=3, framealpha=0)
    centre_circle = plt.Circle((0,0),0.70,fc=face_color)
    ax = fig.gca()
    fig.patch.set_facecolor(face_color)
    ax.add_artist(centre_circle)
    plt.tight_layout()
    plt.savefig(f"Exported/{country}_{'figure_2'}")
    plt.show()

In [ ]:
plot_fig_1("Canada")
plot_fig_2("Denmark")

In [ ]:
def plot_all_figs():
    for country in df_1.index:
        plot_fig_1(country)

    for country in df_2.index:
        plot_fig_2(country)

In [ ]:
""" SAVING THE FIGURES """
plot_all_figs()

### Stacked bar chart

In [ ]:
df_2

In [ ]:
""" do the same for df_1"""

df_2_share = df_2.copy()
for col in df_2_share:
    df_2_share[col] /= df_2.sum(axis=1)
df_2_share

In [ ]:
df_1_share = df_1.copy()
for col in df_1_share:
    df_1_share[col] /= df_1.sum(axis=1)

df_2_share = df_2.copy()
for col in df_2_share:
    df_2_share[col] /= df_2.sum(axis=1)

In [ ]:
def plot_compare_1():
    fig = plt.figure()
    df_1_share.plot(kind='bar', stacked=True, figsize=(bar_size), color=colors_1)
    fig.patch.set_facecolor(face_color)
    plt.legend(bbox_to_anchor=(1.15, 0.4),  labels=labels_1, prop=legend_props, loc = 9, ncol=1, framealpha=0)
    plt.grid(alpha=0.4, axis='y')
    plt.title(f"Comparing countries for heating fuel share", fontdict=title_props, y =1.05)
    plt.xlabel(xlabel='', fontdict=text_props)
    plt.xticks(rotation=45, fontfamily=font, fontsize=20)
    plt.ylabel('Share', fontfamily=font, fontsize=20)
    plt.yticks(fontfamily=font, fontsize=20)
    plt.margins(y=0)
    plt.tight_layout()
    plt.savefig(f"Exported/comparing_countries_fig_1")
    plt.show()

In [ ]:
def plot_compare_2():
    fig = plt.figure()
    df_2_share.plot(kind='bar', stacked=True, figsize=(bar_size), color=colors_2)
    fig.patch.set_facecolor(face_color)
    plt.legend(bbox_to_anchor=(1.15, 0.4), labels=labels_2, prop=legend_props, loc = 9, ncol=1, framealpha=0)
    plt.grid(alpha=0.4, axis='y')
    plt.title(f"Comparing countries for clean heat share", fontdict=title_props, y =1.05)
    plt.xlabel(xlabel='', fontdict=text_props)
    plt.xticks(rotation=45, fontfamily=font, fontsize=20)
    plt.ylabel('Share', fontfamily=font, fontsize=20)
    plt.yticks(fontfamily=font, fontsize=20)
    plt.margins(y=0)
    plt.tight_layout()
    plt.savefig(f"Exported/comparing_countries_fig_2")
    plt.show()

In [ ]:
plot_compare_1()
plot_compare_2()

In [ ]:
ax = plt.gca()
handles, labels = ax.get_legend_handles_labels()